In [ ]:
import time
import mrcfile
import numpy as np
import matplotlib.pyplot as plt

import fsc_utils as fsc

In [ ]:
###

# Example 1
# EMD 24822 
# 20S proteasome
# you will need the half maps from https://www.ebi.ac.uk/emdb/EMD-24822 

###

In [ ]:
map_1_file = ''
map_2_file = ''

with mrcfile.open(map_1_file) as mrc:
    y1 = mrc.data
    voxel = mrc.voxel_size.x
    mrc.close()
    
with mrcfile.open(map_2_file) as mrc:
    y2 = mrc.data
    mrc.close()    

In [ ]:
shape = y1.shape
N = shape[0]
r = N // 2
r_corner = int(np.ceil(np.sqrt(np.sum([s**2 for s in shape]))/2))
rdists = fsc.radial_distance_grid(y1.shape)  

r_signal = 115  # input radius of the structure to be masked for noise estimate

sphere_mask = fsc.sphere_mask(rdists, r_signal)

eps1 = y1 * ~sphere_mask  # estimate the noise from the excluded region
eps2 = y2 * ~sphere_mask

ratio = np.sum(~sphere_mask) / y1.size

freq = fsc.get_radial_spatial_frequencies(y1, voxel)

In [ ]:
### compute the spherically averaged power spectrum ###

y1_raps = fsc.compute_spherically_averaged_power_spectrum(y1, r)
eps1_raps = fsc.compute_spherically_averaged_power_spectrum(eps1, r)

y2_raps = fsc.compute_spherically_averaged_power_spectrum(y2, r)
eps2_raps = fsc.compute_spherically_averaged_power_spectrum(eps2, r)

In [ ]:
### compute the FSC and SFSC ###

t = time.time()
fsc_y1y2 = fsc.two_volume_fsc(y1, y2, rmax=r)
print('FSC: ', time.time() - t)

t = time.time()
y1_whitened = fsc.whitening_transform(y1, eps1, r_corner, ratio) 
y1_upsample = fsc.fourier_upsample(y1_whitened, 2)
sfsc_y1_upsample = np.mean(fsc.single_volume_fsc(y1_upsample, rmax=r, n_splits=1, whiten_upsample=True), axis=0)
print('SFSC 1: ', time.time() - t)

t = time.time()
y2_whitened = fsc.whitening_transform(y2, eps2, r_corner, ratio) 
y2_upsample = fsc.fourier_upsample(y2_whitened, 2)
sfsc_y2_upsample = np.mean(fsc.single_volume_fsc(y2_upsample, rmax=r, n_splits=1, whiten_upsample=True), axis=0)
print('SFSC 2: ', time.time() - t)

# # or the following approximately equal alternative
# ssnr_estimate = (y1_raps - eps1_raps/ratio) / (eps1_raps/ratio)
# fsc_from_ssnr = ssnr_estimate / (1 + ssnr_estimate)

In [ ]:
fsc_res = fsc.linear_interp_resolution(fsc_y1y2, freq)
sfsc_y1_res = fsc.linear_interp_resolution(sfsc_y1_upsample[1:], freq[1:])
sfsc_y2_res = fsc.linear_interp_resolution(sfsc_y2_upsample[1:], freq[1:])

In [ ]:
l1 = 'FSC'
l2 = 'SFSC (half 1)'
l3 = 'SFSC (half 2)'

plt.figure(figsize=(6,4))
plt.plot(freq, fsc_y1y2, color='k', linewidth=2, label=l1)
plt.plot(freq, sfsc_y1_upsample, linewidth=2, label=l2)
plt.plot(freq, sfsc_y2_upsample, linestyle='--', linewidth=2, label=l3)
plt.axhline(1/7, color='gray', linestyle='--')
plt.rc('axes', labelsize=12)
plt.rc('xtick', labelsize=10)
plt.rc('ytick', labelsize=10)
plt.rc('legend', fontsize=10)
plt.grid(which='major', linestyle='--')
plt.xlabel('spatial frequency 'r'(${\AA}^{-1}$)')
plt.title('Fourier shell correlation')
plt.legend()
# plt.savefig('')
plt.show()

In [ ]:
ortho_slice_plot(y1)
ortho_slice_plot(fsc.log_abs(fsc.ftn(y1)))
ortho_slice_plot(eps1)
ortho_slice_plot(fsc.log_abs(fsc.ftn(eps1)))

plt.figure(figsize=(7,4))
plt.semilogy(freq, y1_raps, linewidth=2, label='half 1')
plt.semilogy(freq, eps1_raps, linewidth=2, label='eps 1')
plt.rc('axes', labelsize=12)
plt.rc('xtick', labelsize=10)
plt.rc('ytick', labelsize=10)
plt.rc('legend', fontsize=10)
plt.grid(which='major', linestyle='--')
plt.xlabel('spatial frequency 'r'(${\AA}^{-1}$)')
plt.title('spherically averaged power spectrum')
plt.legend()
# plt.savefig('')
plt.show()

In [ ]:
###

# Example 2
# EMD 13234 
# 70S ribosome
# you will need the half maps from https://www.ebi.ac.uk/emdb/EMD-13234

###

In [ ]:
map_1_file = ''
map_2_file = ''

with mrcfile.open(map_1_file) as mrc:
    y1 = mrc.data
    voxel = mrc.voxel_size.x
    mrc.close()
    
with mrcfile.open(map_2_file) as mrc:
    y2 = mrc.data
    mrc.close() 

In [ ]:
shape = y1.shape
N = shape[0]
r = N // 2
r_corner = int(np.ceil(np.sqrt(np.sum([s**2 for s in shape]))/2))
rdists = fsc.radial_distance_grid(y1.shape) 

r_signal = 95 
r_filled = 160  # radius of unmasked box from emdb

sphere_mask = fsc.sphere_mask(rdists, r_signal)
filled_mask = fsc.sphere_mask(rdists, radius=r_filled)  

eps1 = y1 * ~sphere_mask
eps2 = y2 * ~sphere_mask

ratio = (np.sum(filled_mask) - np.sum(sphere_mask)) / np.sum(filled_mask)

freq = fsc.get_radial_spatial_frequencies(y1, voxel)

In [ ]:
### compute the spherically averaged power spectrum ###

y1_raps = fsc.compute_spherically_averaged_power_spectrum(y1, r)
eps1_raps = fsc.compute_spherically_averaged_power_spectrum(eps1, r)

y2_raps = fsc.compute_spherically_averaged_power_spectrum(y2, r)
eps2_raps = fsc.compute_spherically_averaged_power_spectrum(eps2, r)

In [ ]:
### compute the FSC and SFSC ###

t = time.time()
fsc_y1y2 = fsc.two_volume_fsc(y1, y2, rmax=r)
print('FSC: ', time.time() - t)

t = time.time()
y1_whitened = fsc.whitening_transform(y1, eps1, r_corner, ratio) 
y1_upsample = fsc.fourier_upsample(y1_whitened, 2)
sfsc_y1_upsample = np.mean(fsc.single_volume_fsc(y1_upsample, rmax=r, n_splits=1, whiten_upsample=True), axis=0)
print('SFSC 1: ', time.time() - t)

t = time.time()
y2_whitened = fsc.whitening_transform(y2, eps2, r_corner, ratio) 
y2_upsample = fsc.fourier_upsample(y2_whitened, 2)
sfsc_y2_upsample = np.mean(fsc.single_volume_fsc(y2_upsample, rmax=r, n_splits=1, whiten_upsample=True), axis=0)
print('SFSC 2: ', time.time() - t)

In [ ]:
fsc_res = fsc.linear_interp_resolution(fsc_y1y2, freq)
sfsc_y1_res = fsc.linear_interp_resolution(sfsc_y1_upsample[1:], freq[1:])
sfsc_y2_res = fsc.linear_interp_resolution(sfsc_y2_upsample[1:], freq[1:])

In [ ]:
l1 = 'FSC'
l2 = 'SFSC (half 1)'
l3 = 'SFSC (half 2)'

plt.figure(figsize=(6,4))
plt.plot(freq, fsc_y1y2, color='k', linewidth=2, label=l1)
plt.plot(freq, sfsc_y1_upsample, linewidth=2, label=l2)
plt.plot(freq, sfsc_y2_upsample, linestyle='--', linewidth=2, label=l3)
plt.axhline(1/7, color='gray', linestyle='--')
plt.rc('axes', labelsize=12)
plt.rc('xtick', labelsize=10)
plt.rc('ytick', labelsize=10)
plt.rc('legend', fontsize=10)
plt.grid(which='major', linestyle='--')
plt.xlabel('spatial frequency 'r'(${\AA}^{-1}$)')
plt.title('Fourier shell correlation')
plt.legend()
# plt.savefig('')
plt.show()

In [ ]:
ortho_slice_plot(y1)
ortho_slice_plot(fsc.log_abs(fsc.ftn(y1)))
ortho_slice_plot(eps1)
ortho_slice_plot(fsc.log_abs(fsc.ftn(eps1)))

plt.figure(figsize=(7,4))
plt.semilogy(freq, y1_raps, linewidth=2, label='half 1')
plt.semilogy(freq, eps1_raps, linewidth=2, label='eps 1')
plt.rc('axes', labelsize=12)
plt.rc('xtick', labelsize=10)
plt.rc('ytick', labelsize=10)
plt.rc('legend', fontsize=10)
plt.grid(which='major', linestyle='--')
plt.xlabel('spatial frequency 'r'(${\AA}^{-1}$)')
plt.title('spherically averaged power spectrum')
plt.legend()
# plt.savefig('')
plt.show()

In [ ]:
###

# Example 3
# EMD 27648
# thyrotropin receptor
# you will need the half maps in https://www.ebi.ac.uk/emdb/EMD-27648

###

In [ ]:
map_1_file = ''
map_2_file = ''

with mrcfile.open(map_1_file) as mrc:
    y1 = mrc.data
    voxel = mrc.voxel_size.x
    mrc.close()
    
with mrcfile.open(map_2_file) as mrc:
    y2 = mrc.data
    mrc.close() 

In [ ]:
shape = y1.shape
N = shape[0]
r = N // 2
r_corner = int(np.ceil(np.sqrt(np.sum([s**2 for s in shape]))/2))
rdists = fsc.radial_distance_grid(y1.shape)  

r_signal = 115  # input radius of the structure to be masked for noise estimate

sphere_mask = fsc.sphere_mask(rdists, r_signal)

eps1 = y1 * ~sphere_mask  # estimate the noise from the excluded region
eps2 = y2 * ~sphere_mask

ratio = np.sum(~sphere_mask) / y1.size

freq = fsc.get_radial_spatial_frequencies(y1, voxel)

In [ ]:
### compute the spherically averaged power spectrum ###

y1_raps = fsc.compute_spherically_averaged_power_spectrum(y1, r)
eps1_raps = fsc.compute_spherically_averaged_power_spectrum(eps1, r)

y2_raps = fsc.compute_spherically_averaged_power_spectrum(y2, r)
eps2_raps = fsc.compute_spherically_averaged_power_spectrum(eps2, r)

In [ ]:
### compute the FSC and SFSC ###

t = time.time()
fsc_y1y2 = fsc.two_volume_fsc(y1, y2, rmax=r)
print('FSC: ', time.time() - t)

t = time.time()
y1_whitened = fsc.whitening_transform(y1, eps1, r_corner, ratio) 
y1_upsample = fsc.fourier_upsample(y1_whitened, 2)
sfsc_y1_upsample = np.mean(fsc.single_volume_fsc(y1_upsample, rmax=r, n_splits=1, whiten_upsample=True), axis=0)
print('SFSC 1: ', time.time() - t)

t = time.time()
y2_whitened = fsc.whitening_transform(y2, eps2, r_corner, ratio) 
y2_upsample = fsc.fourier_upsample(y2_whitened, 2)
sfsc_y2_upsample = np.mean(fsc.single_volume_fsc(y2_upsample, rmax=r, n_splits=1, whiten_upsample=True), axis=0)
print('SFSC 2: ', time.time() - t)

In [ ]:
fsc_res = fsc.linear_interp_resolution(fsc_y1y2, freq)
sfsc_y1_res = fsc.linear_interp_resolution(sfsc_y1_upsample[1:], freq[1:])
sfsc_y2_res = fsc.linear_interp_resolution(sfsc_y2_upsample[1:], freq[1:])

In [ ]:
l1 = 'FSC'
l2 = 'SFSC (half 1)'
l3 = 'SFSC (half 2)'

plt.figure(figsize=(6,4))
plt.plot(freq[2:-1], fsc_y1y2[2:-1], color='k', linewidth=2, label=l1)
plt.plot(freq[2:-1], sfsc_y1_upsample[2:-1], linewidth=2, label=l2)
plt.plot(freq[2:-1], sfsc_y2_upsample[2:-1], linestyle='--', linewidth=2, label=l3)
plt.axhline(1/7, color='gray', linestyle='--')
plt.rc('axes', labelsize=12)
plt.rc('xtick', labelsize=10)
plt.rc('ytick', labelsize=10)
plt.rc('legend', fontsize=10)
plt.grid(which='major', linestyle='--')
plt.xlabel('spatial frequency 'r'(${\AA}^{-1}$)')
plt.title('Fourier shell correlation')
plt.legend()
# plt.savefig('')
plt.show()

In [ ]:
ortho_slice_plot(y1)
ortho_slice_plot(fsc.log_abs(fsc.ftn(y1)))
ortho_slice_plot(eps1)
ortho_slice_plot(fsc.log_abs(fsc.ftn(eps1)))

plt.figure(figsize=(7,4))
plt.semilogy(freq, y1_raps, linewidth=2, label='half 1')
plt.semilogy(freq, eps1_raps, linewidth=2, label='eps 1')
plt.rc('axes', labelsize=12)
plt.rc('xtick', labelsize=10)
plt.rc('ytick', labelsize=10)
plt.rc('legend', fontsize=10)
plt.grid(which='major', linestyle='--')
plt.xlabel('spatial frequency 'r'(${\AA}^{-1}$)')
plt.title('spherically averaged power spectrum')
plt.legend()
# plt.savefig('')
plt.show()

In [ ]:
###

# Example 4
# EMD 20278 
# PAC1 GPCR Receptor complex
# you will need the half maps in https://www.ebi.ac.uk/emdb/EMD-20278

###

In [ ]:
map_1_file = ''
map_2_file = ''

with mrcfile.open(map_1_file) as mrc:
    y1 = mrc.data
    voxel = mrc.voxel_size.x
    mrc.close()
    
with mrcfile.open(map_2_file) as mrc:
    y2 = mrc.data
    mrc.close() 

In [ ]:
shape = y1.shape
N = shape[0]
r = N // 2
r_corner = int(np.ceil(np.sqrt(np.sum([s**2 for s in shape]))/2))
rdists = fsc.radial_distance_grid(y1.shape)  

r_signal = 110  # input radius of the structure to be masked for noise estimate

sphere_mask = fsc.sphere_mask(rdists, r_signal)

eps1 = y1 * ~sphere_mask  # estimate the noise from the excluded region
eps2 = y2 * ~sphere_mask

ratio = np.sum(~sphere_mask) / y1.size

freq = fsc.get_radial_spatial_frequencies(y1, voxel)

In [ ]:
### compute the spherically averaged power spectrum ###

y1_raps = fsc.compute_spherically_averaged_power_spectrum(y1, r)
eps1_raps = fsc.compute_spherically_averaged_power_spectrum(eps1, r)

y2_raps = fsc.compute_spherically_averaged_power_spectrum(y2, r)
eps2_raps = fsc.compute_spherically_averaged_power_spectrum(eps2, r)

In [ ]:
Y1 = fsc.ftn(y1)

for ri in range(1, r):  
    mask = fsc.shell_mask(rdists, ri)
    Y1[mask] = (1 / np.sqrt(eps1_raps[ri])) * Y1[mask]

y1_whitened = fsc.iftn(Y1)

In [ ]:
### compute the FSC and SFSC ###

t = time.time()
fsc_y1y2 = fsc.two_volume_fsc(y1, y2, rmax=r)
print('FSC: ', time.time() - t)

t = time.time()
y1_whitened = fsc.whitening_transform(y1, eps1, r_corner, ratio) 
y1_upsample = fsc.fourier_upsample(y1_whitened, 2)
sfsc_y1_upsample = np.mean(fsc.single_volume_fsc(y1_upsample, rmax=r, n_splits=1, whiten_upsample=True), axis=0)
print('SFSC 1: ', time.time() - t)

t = time.time()
y2_whitened = fsc.whitening_transform(y2, eps2, r_corner, ratio) 
y2_upsample = fsc.fourier_upsample(y2_whitened, 2)
sfsc_y2_upsample = np.mean(fsc.single_volume_fsc(y2_upsample, rmax=r, n_splits=1, whiten_upsample=True), axis=0)
print('SFSC 2: ', time.time() - t)

In [ ]:
fsc_res = fsc.linear_interp_resolution(fsc_y1y2, freq)
sfsc_y1_res = fsc.linear_interp_resolution(sfsc_y1_upsample[1:], freq[1:])
sfsc_y2_res = fsc.linear_interp_resolution(sfsc_y2_upsample[1:], freq[1:])

In [ ]:
l1 = 'FSC'
l2 = 'SFSC (half 1)'
l3 = 'SFSC (half 2)'

plt.figure(figsize=(6,4))
plt.plot(freq[1:], fsc_y1y2[1:], color='k', linewidth=2, label=l1)
plt.plot(freq[1:], sfsc_y1_upsample[1:], linewidth=2, label=l2)
plt.plot(freq[1:], sfsc_y2_upsample[1:], linestyle='--', linewidth=2, label=l3)
plt.plot(freq, sfsc_y1)
plt.axhline(1/7, color='gray', linestyle='--')
plt.rc('axes', labelsize=12)
plt.rc('xtick', labelsize=10)
plt.rc('ytick', labelsize=10)
plt.rc('legend', fontsize=10)
plt.grid(which='major', linestyle='--')
plt.xlabel('spatial frequency 'r'(${\AA}^{-1}$)')
plt.title('Fourier shell correlation')
plt.legend()
# plt.savefig('')
plt.show()

In [ ]:
ortho_slice_plot(y1)
ortho_slice_plot(fsc.log_abs(fsc.ftn(y1)))
ortho_slice_plot(eps1)
ortho_slice_plot(fsc.log_abs(fsc.ftn(eps1)))

plt.figure(figsize=(7,4))
plt.semilogy(freq, y1_raps, linewidth=2, label='half 1')
plt.semilogy(freq, eps1_raps, linewidth=2, label='eps 1')
plt.rc('axes', labelsize=12)
plt.rc('xtick', labelsize=10)
plt.rc('ytick', labelsize=10)
plt.rc('legend', fontsize=10)
plt.grid(which='major', linestyle='--')
plt.xlabel('spatial frequency 'r'(${\AA}^{-1}$)')
plt.title('spherically averaged power spectrum')
plt.legend()
# plt.savefig('')
plt.show()